In [1]:
import gzip
import re

In [2]:
with gzip.open("../data/poczatki_wikipediowe.txt.gz", "rt") as f:
    file_content = f.read()

In [3]:
file_content.count("### ")

1008143

In [4]:
file_content.count("\n") / 3

1008135.6666666666

In [6]:
d = {}
for part in file_content.removeprefix("### ").split("\n### "):
    splited = part.strip().split("\n", maxsplit=1)
    if len(splited) == 2:
        title, content = splited
        d[title] = content

In [17]:
sum(v.startswith("'''") for v in d.values())

561805

In [167]:
d["Wahadłowiec kosmiczny"]

"'''Wahadłowiec kosmiczny''', '''prom kosmiczny''' - rodzaj załogowego '''Wahadłowiec kosmiczny''', '''prom kosmiczny''' - rodzaj załogowego statku kosmicznego, który może być wykorzystywany wielokrotnie i zwykle przystosowany jest do wynoszenia na orbitę i ściągania z orbity sztucznych satelitów i innych ładunków. W założeniach prom, dzięki możliwości"

In [190]:
basic_pattern = re.compile(r"'''([\w\s]+)'''(?:, '''([\w\s]+)''')*")
basic_extract_pattern = re.compile(r"'''(?<!,)([\w\s]+)'''")

In [191]:
another_pattern = re.compile(r"'''([\w\s]+)'''(?:, '''([\w\s]+)''')*(?: lub '''([\w\s]+)''')*")

In [251]:
counter = 0

for k, v in d.items():
    if v.startswith("* "):
        # enumeration
        pass
    if (m1:= another_pattern.match(v)) and (m2:= basic_pattern.match(v)):
        if m1.group() != m2.group():
            if len(basic_extract_pattern.findall(m2.group())) > 1:
                print(v)
                counter += 1
                if counter > 20:
                    break

'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie''' (dosł. leśni ludzie ) – lud germański, odłam Gotów. W IV wieku przyjęli arianizm (dzięki Biblii przetłumaczonej przez Wulfilę na język gocki). = = Wczesne dzieje = = Pierwsze przekazy traktujące Wizygotów jako osobny lud odwołują się do roku 268, w którym najechali oni cesarstwo rzymskie i opanowali Półwysep
'''Reakcja termojądrowa''', '''synteza jądrowa''' lub '''fuzja jądrowa''' – '''Reakcja termojądrowa''', '''synteza jądrowa''' lub '''fuzja jądrowa''' – zjawisko polegające na złączeniu się dwóch lżejszych jąder w jedno cięższe, w wyniku fuzji mogą powstawać obok nowych jąder też wolne
'''Alcuin''', '''Alkuin''', '''Alchoin''', '''Alcwin''', '''Ealhwine''', '''Alchwine''' lub '''Albinus Flaccus''' (ur. ok. '''Alcuin''', '''Alkuin''', '''Alchoin''', '''Alcwin''', '''Ealhwine''', '''Alchwine''' lub '''Albinus
'''Pucz moskiewski''', '''Pucz sierpniowy''' lub '''Pucz Janajewa''' to nieudana próba przejęcia władzy w ZSRR przez twa

In [235]:
another_pattern = re.compile(r"'''([\w\s]+)'''(?:, '''([\w\s]+)''')*(?: lub '''([\w\s]+)''')*")

def extract_synonyms(title, content):
    if content.startswith("* "):
        # enumeration
        return set()
    elif m := another_pattern.match(content):
        return set(basic_extract_pattern.findall(m.group()))
    else:
        return set()

In [237]:
with open("out", "wt") as f:
    for k, v in d.items():
        if v.startswith("* "):
            # enumeration
            continue

        synonyms = extract_synonyms(k, v)
        if len(synonyms) > 1:
            f.write(" # ".join(synonyms) + "\n")

In [213]:
extract_synonyms("Endre Kelemen", d["Endre Kelemen"])

set()

In [219]:
d["Zabytki Głogowa"]

"'''Głogów''' jest jednym z najstarszych '''Głogów''' jest jednym z najstarszych miast w Polsce. Pierwotnie znajdował się tu gród plemienia Dziadoszan, później na terenie Ostrowa Tumskiego Mieszko I zbudował nowy gród. Do historii przeszła bohaterska obrona grodu przed Niemcami w roku 1109. Spalony w roku 1157 przez Fryderyka I Barbarossę i znów odbudowany, stał się"

In [241]:
idx = file_content.find("ujezd wasylkowski")
file_content[idx - 100: idx + 1000]

" zewnętrzne = = 553 miasto * winnicki Kategoria:Winnica\n\n### Powiat wasylkowski\n'''Powiat''' lub '''ujezd wasylkowski''' dawny powiat, położony w środkowej części guberni kijowskiej. Siedzibą władz było miasto Wasylków. Opis powiatu wasylkowskiego pod względem historycznym, obyczajowym i statystycznym wydał Edward Rulikowski (Warsz., 1853). = = Linki zewnętrzne = = * *\n\n### Humanizm\n'''Humanizm''' – prąd filozoficzny, etyczny i kulturowy, uznający człowieka za najwyższą wartość i podkreślający jego godność. Wielu humanistów (w tym ludzi wierzących) postuluje rozstrzyganie dylematów etycznych poprzez odwoływanie się do pojęć uniwersalnych, wspólnych wszystkim ludziom. Poszukują oni źródła prawdy i moralności w człowieku i jego dążeniu\n\n### Powiat władysławowski\n'''Powiat''' lub '''ujezd władysławowski''' dawny powiat guberni suwalskiej. Siedzibą powiatu był Władysławów na Litwie. = = Wzmianka z 1893 = = Powiat guberni suwalskiej, utworzony w 1867 z zachodniej połowy dawnego po

In [239]:
d["ujezd władysławowski"]

KeyError: 'ujezd władysławowski'

In [253]:
useful = [
    "Wizygoci", "Emotikon", "Reakcja termojądrowa", "Zabytki Głogowa", "Powiat wasylkowski"
]
for title in useful:
    print(title, d[title], "Extracted: " + str(extract_synonyms(title, d[title])), sep="\n", end="\n\n")

Wizygoci
'''Wizygoci''', '''Goci zachodni''' lub '''Terwingowie''' (dosł. leśni ludzie ) – lud germański, odłam Gotów. W IV wieku przyjęli arianizm (dzięki Biblii przetłumaczonej przez Wulfilę na język gocki). = = Wczesne dzieje = = Pierwsze przekazy traktujące Wizygotów jako osobny lud odwołują się do roku 268, w którym najechali oni cesarstwo rzymskie i opanowali Półwysep
Extracted: {'Terwingowie', 'Goci zachodni', 'Wizygoci'}

Emotikon
'''Emotikon''' lub '''emotikona''' (ang. ''emoticon''), '''uśmieszek''' (''smile'', ''smiley''), '''buźka''' – złożony ze znaków tekstowych (ASCII) wyraz nastroju, używany przez użytkowników Internetu. Najczęściej przedstawia symboliczny ludzki grymas twarzy, obrócony o 90° w kierunku przeciwnym do wskazówek zegara, w licznych wariantach. Niektóre programy jak np.
Extracted: {'Emotikon', 'emotikona'}

Reakcja termojądrowa
'''Reakcja termojądrowa''', '''synteza jądrowa''' lub '''fuzja jądrowa''' – '''Reakcja termojądrowa''', '''synteza jądrowa''' lub '